In [1]:
# import sys
# sys.path.append('./utils/')
# from leetcode_process_utils import *

import sys
sys.path.append('./utils/')
from leetcode_exec_utils import *


In [2]:
apps_data_path = "/home/mingzhu/CodeModel/apps/apps/APPS/"
apps_train_data = apps_data_path + 'train/'
apps_test_data = apps_data_path + 'test/'

In [3]:
input_redirect_code = """
import sys
from io import StringIO
sys.stdin = StringIO(inputs)
inputs_line_iterator = iter(inputs.split("\\n"))
sys.stdin.readline = lambda *args: next(inputs_line_iterator)
sys.stdin.readlines = lambda *args: inputs.split("\\n")
sys.stdin.read = lambda *args: inputs
"""
imports = """import sys\nimport time\nimport itertools
from itertools import accumulate, product, permutations, combinations\n
import collections
from collections import Counter, OrderedDict, deque, defaultdict, ChainMap
from functools import lru_cache\nimport math
from math import sqrt, sin, cos, tan, ceil, fabs, floor, gcd, exp, log, log2
import fractions\nfrom typing import List, Tuple\nimport numpy as np
import random\nimport heapq\nfrom heapq import *\n"""

In [ ]:
# input_output.json  metadata.json  question.txt  solutions.json  starter_code.py


In [5]:
def get_apps_dict(apps_path):
    fns = os.listdir(apps_path)
    apps_train_dict = {}
    fail_list = []
    for fn in fns:
        prob_dict = {}
        prob_path = apps_path + fn + '/'
        io_file = "input_output.json"
        level_file = "metadata.json"
        desc_file = "question.txt"
        sol_file = "solutions.json"
        code_file = "starter_code.py"
        json_files = [io_file, level_file, sol_file]
        txt_files = [desc_file, code_file]
        all_files = json_files + txt_files
        js_list = []
        for jf in json_files:
            if not os.path.isfile(prob_path + jf):
                js_list.append(None)
                continue
            with open(prob_path + jf) as infile:
                js_list.append(json.load(infile))
        txt_list = []
        for txt in txt_files:
            if not os.path.isfile(prob_path + txt):
                txt_list.append(None)
                continue
            with open(prob_path + txt) as infile:
                txt_list.append(infile.read())
        prob_dict['idx'] = fn
        prob_dict['io'] = js_list[0]
        prob_dict['level'] = js_list[1]
        prob_dict['solutions'] = js_list[2]
        prob_dict['desc'] = txt_list[0]
        prob_dict['starter_code'] = txt_list[1]
        apps_train_dict[fn] = prob_dict
    return apps_train_dict

def run_exec_python_test(codestring, timeout=5):
    fn_name = tmp_path + "test" + '.py'
    f = open(fn_name, 'w')
    f.write(codestring)
    f.close()
    cmd = "python3 " + fn_name
    output = run_command(cmd, timeout)
    return output



In [6]:
apps_train_dict = get_apps_dict(apps_train_data)
apps_test_dict = get_apps_dict(apps_test_data)

In [7]:
len(apps_train_dict)

5000

In [8]:
len(apps_test_dict)

5000

In [ ]:
count = 0
for key, dic in apps_train_dict.items():
    if dic["io"]:
        if 'fn_name' in dic["io"]:
            print(key)
#             break
            count += 1

In [150]:
key = '0719'
dic = apps_train_dict[key]
sol = dic['solutions'][0]
test_io = dic['io']
test_inputs = test_io['inputs']
func_name = ""
if 'fn_name' in test_io:
    func_name = test_io['fn_name']
print(test_inputs)
a = get_apps_callcode(dic)
# inputs = "\"" + "\\n".join(test_inputs[0]) + "\""

[['3', '1 5', '10 3', '3 4']]


In [ ]:
print(a)

In [146]:
# a = "import sys\nprint(sys.version)\n" + a
b = run_exec_python_test(a)
print(b)

60
1
120


In [143]:
def get_apps_callcode(dic):
    sol = dic['solutions'][0]
    test_io = dic['io']
    if not test_io:
        return None
    test_inputs = test_io['inputs']
    if len(test_inputs) == 0:
        return None
    func_name = ""
    a = imports
    if 'fn_name' in test_io:
        func_name = test_io['fn_name']
        inputs = str(test_inputs[0]).replace('\n', '\\n').replace('\t', '\\t')
        input_line = "\ninputs=" + inputs + '\n'
        if "class Solution" in sol:
            output_line = "output = Solution()." + func_name + "(*inputs)\n"
        else:
            output_line = "output = " + func_name + "(*inputs)\n"
        a += sol + input_line + output_line
        a += "print(output)\n"
    else:
        if type(test_inputs[0]) == list:
            inputs = "\"" + "\\n".join(test_inputs[0]) + "\""
        else:
            inputs = "\"" + test_inputs[0].replace('\n', '\\n').replace('\t', '\\t') + "\""
        a += "inputs=" + inputs + "\n" + input_redirect_code + sol
    return a

programs = []
pids = []
program_id_dict = {}
for key, dic in apps_train_dict.items():
    a = get_apps_callcode(dic)
    if a:
        programs.append(a)  
        pids.append(key)
        program_id_dict[key] = [len(programs)-1]

In [133]:
len(programs)

4450

In [147]:
import time
from tqdm.auto import tqdm
from p_tqdm import p_map, p_umap, p_imap, p_uimap

py_results = p_map(run_exec_python3, programs)

In [148]:
lang = "Python"
result_dict = {lang:py_results}
result_type_dict = show_result_summary(result_dict)
result_id_dict, pass_list, fail_list, error_type_dict = get_pass_fail_error(py_results, 
                                                                            program_id_dict, pids, lang)
print(len(pass_list), len(fail_list))
error_type_dict

Python error 80 0.017977528089887642
Python timeout 14 0.003146067415730337
Python empty 40 0.008988764044943821
Python other 6 0.001348314606741573
Python good 4310 0.9685393258426966
4310 140


{'AttributeError': 1,
 'ModuleNotFoundError': 2,
 'Error': 2,
 'ImportError': 2,
 'NameError': 2,
 'TypeError': 3,
 'IndexError': 3,
 'EOFError': 5,
 'ValueError': 12,
 'SyntaxError': 44}

In [ ]:
for i, idx in enumerate(fail_list):
    for error in result_id_dict[idx]:
        if "SyntaxError" in error:
            print(idx, error)